In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import PIL
from PIL import Image
from sklearn.metrics import accuracy_score

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


# 1. Get Data

In [ ]:
def get_data(data_set, label_column='label'):
    # data_set:   pd DataFrame
    data_X = data_set.drop(columns=label_column)
    data_y = data_set[label_column]
    return data_X, data_y

In [ ]:
# read data
train_set = pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv')
test_set = pd.read_csv('/kaggle/input/Kannada-MNIST/test.csv')
dig_set = pd.read_csv('/kaggle/input/Kannada-MNIST/Dig-MNIST.csv')

In [ ]:
X_train, y_train = get_data(train_set, label_column='label')
X_dig, y_dig = get_data(dig_set, label_column='label')

In [ ]:
X_train.shape, y_train.shape, X_dig.shape, y_dig.shape

In [ ]:
X_train = X_train.to_numpy().reshape(-1, 28, 28, 1)
y_train = y_train.to_numpy().reshape(-1, 1)
X_dig = X_dig.to_numpy().reshape(-1, 28, 28, 1)
y_dig = y_dig.to_numpy().reshape(-1, 1)

In [ ]:
test_set = test_set.drop(columns='id').to_numpy().reshape(-1, 28, 28, 1)

In [ ]:
# no need to do this for CNN, otherwise some image preprocessing packages 
# can convert any value between 0-1 to 0 or 1
# X_train = X_train / 255.0
# X_dig = X_dig / 255.0
# test_set = test_set / 255.0

In [ ]:
y_train_cat = to_categorical(y_train)
y_dig_cat = to_categorical(y_dig)

In [ ]:
class_names = np.arange(10)

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i,:,:,0], cmap='gray')
    # y_train_cat is in one-hot format
    plt.xlabel(np.where(y_train_cat[i]==1)[0][0])
plt.show()

# 2. array to rgb

In [ ]:
def array2rgb(input_array, reshape_size, interp_method):
    output_list = []
    for i in range(len(input_array)):
        data = input_array[i,:,:, 0]
        img_rgb = Image.fromarray(data.astype('uint8')).convert('RGB')
        img_resized = img_rgb.resize(reshape_size, resample=interp_method)
        img_resized = np.array(img_resized).astype('float32')
        output_list.append(img_resized)
    output_array = np.array(output_list)

    return output_array

In [ ]:
reshape_size = (94, 94)
interp_method = PIL.Image.BICUBIC

X_train_rgb = array2rgb(X_train, reshape_size=reshape_size, interp_method=interp_method)
X_dig_rgb = array2rgb(X_dig, reshape_size=reshape_size, interp_method=interp_method)
test_set_rgb = array2rgb(test_set, reshape_size=reshape_size, interp_method=interp_method)

In [ ]:
X_train_rgb.shape, X_dig_rgb.shape, test_set_rgb.shape

In [ ]:
class_names = np.arange(10)

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train_rgb[i,:,:,:], cmap='gray')
    # y_train_cat is in one-hot format
    plt.xlabel(np.where(y_train_cat[i]==1)[0][0])
plt.show()

# 3. data generator

In [ ]:
height_shift_range = 0.1
width_shift_range = 0.1
zoom_range = 0.1
rotation_range=10
datagen = ImageDataGenerator(rotation_range=10, zoom_range=zoom_range,
                             width_shift_range=width_shift_range,
                             height_shift_range=height_shift_range)

In [ ]:
it = datagen.flow(X_train_rgb[0:10,:,:,:], batch_size=1)
plt.figure(figsize=(5, 5))
for i in range(9):
    plt.subplot(330 + 1+ i)
    batch = it.next()
    image = batch[0].astype('uint8')
    #print(image.shape)
    plt.imshow(image, cmap='gray')



# 3. build model

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    weights = 'imagenet', input_shape = (94, 94, 3),
    include_top = False
)

base_model.trainable =False

In [ ]:
model = models.Sequential([
    base_model, layers.GlobalAveragePooling2D()
    ,
    layers.Dropout(0.4), layers.Dense(10, activation='softmax')
])

"""
# Another way to implement the model
inputs = tf.keras.Input(shape=(94, 94, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)
"""
print()

In [ ]:
# Adam optimizer
learning_rate = 0.001
beta_1 = 0.9
beta_2 = 0.999
optimizer = Adam(lr=learning_rate, beta_1=beta_1, beta_2=beta_2)

In [ ]:
model.compile(optimizer=optimizer,
              loss=['categorical_crossentropy'],
              metrics=['accuracy'])

In [ ]:
batch_size = 32
# reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,
                              patience=3, min_lr=0.00001)
history = model.fit(datagen.flow(X_train_rgb, y_train_cat, batch_size=batch_size), callbacks=[reduce_lr],
          steps_per_epoch=len(X_train) / batch_size, epochs=10, validation_data=(X_dig_rgb, y_dig_cat))

In [ ]:
history.history

In [ ]:
plt.plot(history.history['acc'], label='accuracy')
plt.plot(history.history['val_acc'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
start= time.time()
y_pred = model.predict_classes(test_set_rgb)
end = time.time()
test_time = (end - start) / 60
print(f'testing time: {test_time}')

In [ ]:
y_pred_pd = pd.DataFrame({'id': np.arange(len(y_pred)),
                          'label': y_pred})
y_pred_pd.to_csv('./y_pred.csv', sep=',', index=False)

# 4. compare to the previous model

In [ ]:
# conv layers
model_2 = models.Sequential()
model_2.add(layers.Conv2D(32, kernel_size=(3, 3),padding='Same', activation='relu', input_shape=(28, 28, 1)))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Conv2D(32, kernel_size=(3, 3), padding='Same', activation='relu'))
model_2.add(layers.BatchNormalization(momentum=0.1))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Dropout(0.2))

model_2.add(layers.Conv2D(64, kernel_size=(5, 5),padding='Same', activation='relu', input_shape=(28, 28, 1)))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Conv2D(64, kernel_size=(5, 5), padding='Same', activation='relu'))
model_2.add(layers.BatchNormalization(momentum=0.1))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Dropout(0.2))

"""
model_2.add(layers.Conv2D(32, kernel_size=(3, 3),padding='Same', activation='relu', input_shape=(28, 28, 1)))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Conv2D(32, kernel_size=(3, 3), padding='Same', activation='relu'))
model_2.add(layers.BatchNormalization(momentum=0.1))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Dropout(0.2))
"""

# Fully connected layers
model_2.add(layers.Flatten())
model_2.add(layers.Dense(128, activation='relu'))
model_2.add(layers.Dropout(0.4))
model_2.add(layers.Dense(64, activation='relu'))
model_2.add(layers.Dropout(0.4))
model_2.add(layers.Dense(10, activation='softmax'))

In [ ]:
model_2.compile(optimizer=optimizer,
              loss=['categorical_crossentropy'],
              metrics=['accuracy'])

In [ ]:

history_2 = model_2.fit(datagen.flow(X_train, y_train_cat, batch_size=batch_size), callbacks=[reduce_lr],
          steps_per_epoch=len(X_train) / batch_size, epochs=10, validation_data=(X_dig, y_dig_cat))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10,5))
axs[0].plot(history_2.history['acc'], label='accuracy')
axs[0].plot(history_2.history['val_acc'], label = 'val_accuracy')
axs[0].set_ylim(0.5, 1)
axs[0].set_title('transfer learning: MobileNetV2')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy')
axs[0].legend(loc='lower right')

axs[1].plot(history.history['acc'], label='accuracy')
axs[1].plot(history.history['val_acc'], label = 'val_accuracy')
axs[1].set_ylim(0.5, 1)
axs[1].set_title('classic CNN')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
axs[1].legend(loc='lower right')

It seems that the classic CNN model works better, we might need to unfreeze the weights of the MobileNetV2 model to achieve similar or better result.